<div align="center">
<img style="display: block; margin: auto;" alt="photo" src="https://cdn.quantconnect.com/web/i/icon.png"> <img style="display: block; margin: auto;" alt="photo" src="https://www.marketing-branding.com/wp-content/uploads/2020/07/google-colaboratory-colab-guia-completa.jpg " width="50" height="50">
<img style="display: block; margin: auto;" alt="photo" src="https://upload.wikimedia.org/wikipedia/commons/d/da/Yahoo_Finance_Logo_2019.svg" width="50" height="50">  

Quantconnect -> Google Colab with Yahoo Finance data

Introduction to Financial Python
</div>

# 05 Pandas-Resampling and DataFrame

# Introduction
In the last chapter we had a glimpse of Pandas. In this chapter we will learn about resampling methods and the DataFrame object, which is a powerful tool for financial data analysis.

# Fetching Data
Here we use the Yahoo Finance to retrieve data.


In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf

aapl = yf.Ticker("AAPL")

# get stock info
print(aapl.info)

# get historical market data
aapl_table = aapl.history(start="2016-01-01",  end="2017-12-31")
aapl_table

{'zip': '95014', 'sector': 'Technology', 'fullTimeEmployees': 147000, 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, iPod touch, and other Apple-branded and third-party accessories. It also provides AppleCare support services; cloud services store services; and operates various platforms, including the App Store, that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Music, which offers users a curated listening experience with on-demand radi

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2016-01-04,23.655164,24.291441,23.514538,24.286829,270597600,0.0,0
2016-01-05,24.379048,24.402102,23.609063,23.678223,223164000,0.0,0
2016-01-06,23.182569,23.599838,23.023501,23.214844,273829600,0.0,0
2016-01-07,22.749162,23.083437,22.230459,22.235069,324377600,0.0,0
2016-01-08,22.719195,22.848294,22.306538,22.352644,283192000,0.0,0
...,...,...,...,...,...,...,...
2017-12-22,41.827756,42.004953,41.784656,41.906776,65397600,0.0,0
2017-12-26,40.898678,41.059112,40.630488,40.843605,132742000,0.0,0
2017-12-27,40.731058,40.893884,40.637671,40.850784,85992800,0.0,0


We will create a Series named "aapl" whose values are Apple's daily closing prices, which are of course indexed by dates:

In [ ]:
aapl = aapl_table['Close']['2017']

In [ ]:
print(aapl)

Date
2017-01-03    27.372358
2017-01-04    27.341724
2017-01-05    27.480768
2017-01-06    27.787128
2017-01-09    28.041649
                ...    
2017-12-22    41.906776
2017-12-26    40.843605
2017-12-27    40.850784
2017-12-28    40.965721
2017-12-29    40.522728
Name: Close, Length: 251, dtype: float64


Recall that we can fetch a specific data point using series['yyyy-mm-dd']. We can also fetch the data in a specific month using series['yyyy-mm'].

In [ ]:
print(aapl['2017-3'])

Date
2017-03-01    33.086288
2017-03-02    32.889832
2017-03-03    33.083923
2017-03-06    32.979778
2017-03-07    33.022381
2017-03-08    32.899300
2017-03-09    32.823563
2017-03-10    32.932438
2017-03-13    32.946632
2017-03-14    32.896942
2017-03-15    33.244869
2017-03-16    33.299309
2017-03-17    33.133629
2017-03-20    33.481548
2017-03-21    33.098129
2017-03-22    33.472088
2017-03-23    33.353745
2017-03-24    33.287468
2017-03-27    33.344273
2017-03-28    34.035397
2017-03-29    34.111137
2017-03-30    34.066166
2017-03-31    34.002254
Name: Close, dtype: float64


In [ ]:
aapl['2017-2':'2017-4']

Date
2017-02-01    30.341728
2017-02-02    30.289881
2017-02-03    30.419493
2017-02-06    30.704649
2017-02-07    30.996874
                ...    
2017-04-24    33.997528
2017-04-25    34.208176
2017-04-26    34.006981
2017-04-27    34.033031
2017-04-28    33.999889
Name: Close, Length: 61, dtype: float64

.head(N) and .tail(N) are methods for quickly accessing the first or last N elements.

In [ ]:
print(aapl.head(5))
print(aapl.tail(10))

Date
2017-01-03    27.372358
2017-01-04    27.341724
2017-01-05    27.480768
2017-01-06    27.787128
2017-01-09    28.041649
Name: Close, dtype: float64
Date
2017-12-15    41.657734
2017-12-18    42.244408
2017-12-19    41.794231
2017-12-20    41.748730
2017-12-21    41.906776
2017-12-22    41.906776
2017-12-26    40.843605
2017-12-27    40.850784
2017-12-28    40.965721
2017-12-29    40.522728
Name: Close, dtype: float64


# Resampling
**_series.resample(freq)_** is a class called "DatetimeIndexResampler" which groups data in a Series object into regular time intervals. The argument "freq" determines the length of each interval.

**_series.resample.mean()_** is a complete statement that groups data into intervals, and then compute the mean of each interval. For example, if we want to aggregate the daily data into monthly data by mean:

In [ ]:
by_month = aapl.resample('M').mean()
print(by_month)

Date
2017-01-31    28.178332
2017-02-28    31.606277
2017-03-31    33.282221
2017-04-30    33.819265
2017-05-31    36.125683
2017-06-30    35.133983
2017-07-31    35.245241
2017-08-31    37.897228
2017-09-30    37.604737
2017-10-31    37.654550
2017-11-30    41.233916
2017-12-31    41.160038
Freq: M, Name: Close, dtype: float64


We can also aggregate the data by week:

In [ ]:
by_week = aapl.resample('W').mean()
print(by_week.head())

Date
2017-01-08    27.495495
2017-01-15    28.097736
2017-01-22    28.266116
2017-01-29    28.553981
2017-02-05    29.662543
Freq: W-SUN, Name: Close, dtype: float64


We can also aggregate the data by month with max:

In [ ]:
aapl.resample('M').max()

Date
2017-01-31    28.739210
2017-02-28    32.451962
2017-03-31    34.111137
2017-04-30    34.264984
2017-05-31    37.099129
2017-06-30    36.944641
2017-07-31    36.471695
2017-08-31    39.129723
2017-09-30    39.141659
2017-10-31    40.332245
2017-11-30    42.050140
2017-12-31    42.244408
Freq: M, Name: Close, dtype: float64

We can choose almost any frequency by using the format 'nf', where 'n' is an integer and 'f' is M for month, W for week and D for day.

In [ ]:
three_day = aapl.resample('3D').mean()
two_week = aapl.resample('2W').mean()
two_month = aapl.resample('2M').mean()


print(three_day)
print(two_week)
print(two_month )

Date
2017-01-03    27.398284
2017-01-06    27.787128
2017-01-09    28.110777
2017-01-12    28.078175
2017-01-15    28.279665
                ...    
2017-12-17    42.019320
2017-12-20    41.854094
2017-12-23          NaN
2017-12-26    40.886703
2017-12-29    40.522728
Freq: 3D, Name: Close, Length: 121, dtype: float64
Date
2017-01-08    27.495495
2017-01-22    28.172571
2017-02-05    29.108262
2017-02-19    31.516819
2017-03-05    32.595556
2017-03-19    33.017884
2017-04-02    33.625220
2017-04-16    33.849730
2017-04-30    33.791846
2017-05-14    35.697631
2017-05-28    36.496177
2017-06-11    36.515006
2017-06-25    34.529045
2017-07-09    34.265135
2017-07-23    35.278871
2017-08-06    36.272773
2017-08-20    37.904119
2017-09-03    38.458794
2017-09-17    38.282706
2017-10-01    36.840872
2017-10-15    37.071597
2017-10-29    37.757318
2017-11-12    41.032238
2017-11-26    41.204377
2017-12-10    40.867066
2017-12-24    41.620867
2018-01-07    40.795710
Freq: 2W-SUN, Name: Close, 

Besides the mean() method, other methods can also be used with the resampler:



In [ ]:
std = aapl.resample('W').std()
max = aapl.resample('W').max()
min = aapl.resample('W').min()


print(std)
print(max)
print(min)

Date
2017-01-08    0.203367
2017-01-15    0.072531
2017-01-22    0.025552
2017-01-29    0.245284
2017-02-05    0.943259
2017-02-12    0.252004
2017-02-19    0.231392
2017-02-26    0.059342
2017-03-05    0.340091
2017-03-12    0.076286
2017-03-19    0.177837
2017-03-26    0.157260
2017-04-02    0.319808
2017-04-09    0.128689
2017-04-16    0.212476
2017-04-23    0.174676
2017-04-30    0.090026
2017-05-07    0.235639
2017-05-14    0.352979
2017-05-21    0.536092
2017-05-28    0.060389
2017-06-04    0.281220
2017-06-11    0.620034
2017-06-18    0.382566
2017-06-25    0.128894
2017-07-02    0.264111
2017-07-09    0.158892
2017-07-16    0.404280
2017-07-23    0.124988
2017-07-30    0.384442
2017-08-06    0.924387
2017-08-13    0.467469
2017-08-20    0.435253
2017-08-27    0.275767
2017-09-03    0.251727
2017-09-10    0.381637
2017-09-17    0.294141
2017-09-24    0.735189
2017-10-01    0.354847
2017-10-08    0.204859
2017-10-15    0.118731
2017-10-22    0.516998
2017-10-29    0.680457
2017-1

Often we want to calculate monthly returns of a stock, based on prices on the last day of each month. To fetch those prices, we use the series.resample.agg() method:

In [ ]:
last_day = aapl.resample('M').agg(lambda x: x[-1])
print(last_day)

Date
2017-01-31    28.597816
2017-02-28    32.423576
2017-03-31    34.002254
2017-04-30    33.999889
2017-05-31    36.305332
2017-06-30    34.228161
2017-07-31    35.347557
2017-08-31    39.129723
2017-09-30    36.772392
2017-10-31    40.332245
2017-11-30    41.150105
2017-12-31    40.522728
Freq: M, Name: Close, dtype: float64


Or directly calculate the monthly rates of return using the data for the first day and the last day:

In [ ]:
monthly_return = aapl.resample('M').agg(lambda x: x[-1]/x[0] - 1)
print(monthly_return)

Date
2017-01-31    0.044770
2017-02-28    0.068613
2017-03-31    0.027684
2017-04-30   -0.000348
2017-05-31    0.046463
2017-06-30   -0.059799
2017-07-31    0.036446
2017-08-31    0.097261
2017-09-30   -0.060531
2017-10-31    0.099019
2017-11-30    0.033422
2017-12-31   -0.010640
Freq: M, Name: Close, dtype: float64


Series object also provides us some convenient methods to do some quick calculation.

In [ ]:
print(monthly_return.mean())
print(monthly_return.std())
print(monthly_return.max())

0.026863264084197352
0.05225865308159278
0.09901852037224312


Another two methods frequently used on Series are .diff() and .pct_change(). The former calculates the difference between consecutive elements, and the latter calculates the percentage change.

In [ ]:
print(last_day.diff())
print(last_day.pct_change())

Date
2017-01-31         NaN
2017-02-28    3.825760
2017-03-31    1.578678
2017-04-30   -0.002365
2017-05-31    2.305443
2017-06-30   -2.077171
2017-07-31    1.119396
2017-08-31    3.782166
2017-09-30   -2.357330
2017-10-31    3.559853
2017-11-30    0.817860
2017-12-31   -0.627377
Freq: M, Name: Close, dtype: float64
Date
2017-01-31         NaN
2017-02-28    0.133778
2017-03-31    0.048689
2017-04-30   -0.000070
2017-05-31    0.067807
2017-06-30   -0.057214
2017-07-31    0.032704
2017-08-31    0.106999
2017-09-30   -0.060244
2017-10-31    0.096808
2017-11-30    0.020278
2017-12-31   -0.015246
Freq: M, Name: Close, dtype: float64


Notice that we induced a NaN value while calculating percentage changes i.e. returns.

When dealing with NaN values, we usually either removing the data point or fill it with a specific value. Here we fill it with 0:

In [ ]:
daily_return = last_day.pct_change()
print(daily_return.fillna(0))

Date
2017-01-31    0.000000
2017-02-28    0.133778
2017-03-31    0.048689
2017-04-30   -0.000070
2017-05-31    0.067807
2017-06-30   -0.057214
2017-07-31    0.032704
2017-08-31    0.106999
2017-09-30   -0.060244
2017-10-31    0.096808
2017-11-30    0.020278
2017-12-31   -0.015246
Freq: M, Name: Close, dtype: float64


Alternatively, we can fill a NaN with the next fitted value. This is called 'backward fill', or 'bfill' in short:

In [ ]:
daily_return = last_day.pct_change()
print(daily_return.fillna(method = 'bfill'))

Date
2017-01-31    0.133778
2017-02-28    0.133778
2017-03-31    0.048689
2017-04-30   -0.000070
2017-05-31    0.067807
2017-06-30   -0.057214
2017-07-31    0.032704
2017-08-31    0.106999
2017-09-30   -0.060244
2017-10-31    0.096808
2017-11-30    0.020278
2017-12-31   -0.015246
Freq: M, Name: Close, dtype: float64


As expected, since there is a 'backward fill' method, there must be a 'forward fill' method, or 'ffill' in short. However we can't use it here because the NaN is the first value.

We can also simply remove NaN values by **_.dropna()_**

In [ ]:
daily_return = last_day.pct_change()
daily_return.dropna()

Date
2017-02-28    0.133778
2017-03-31    0.048689
2017-04-30   -0.000070
2017-05-31    0.067807
2017-06-30   -0.057214
2017-07-31    0.032704
2017-08-31    0.106999
2017-09-30   -0.060244
2017-10-31    0.096808
2017-11-30    0.020278
2017-12-31   -0.015246
Freq: M, Name: Close, dtype: float64

# DataFrame
The **DataFrame** is the most commonly used data structure in Pandas. It is essentially a table, just like an Excel spreadsheet.

More precisely, a DataFrame is a collection of Series objects, each of which may contain different data types. A DataFrame can be created from various data types: dictionary, 2-D numpy.ndarray, a Series or another DataFrame.

## Create DataFrames
The most common method of creating a DataFrame is passing a dictionary:

In [ ]:
import pandas as pd

dict = {'AAPL': [143.5, 144.09, 142.73, 144.18, 143.77],'GOOG':[898.7, 911.71, 906.69, 918.59, 926.99],
        'IBM':[155.58, 153.67, 152.36, 152.94, 153.49]}
data_index = pd.date_range('2017-07-03',periods = 5, freq = 'D')
df = pd.DataFrame(dict, index = data_index)
print(df)

              AAPL    GOOG     IBM
2017-07-03  143.50  898.70  155.58
2017-07-04  144.09  911.71  153.67
2017-07-05  142.73  906.69  152.36
2017-07-06  144.18  918.59  152.94
2017-07-07  143.77  926.99  153.49


## Manipulating DataFrames
We can fetch values in a DataFrame by columns and index. Each column in a DataFrame is essentially a Pandas Series. We can fetch a column by square brackets: **df['column_name']**

If a column name contains no spaces, then we can also use df.column_name to fetch a column:

In [ ]:
df = aapl_table
print(df.Close.tail(5))
print(df['Volume'].tail(5))

Date
2017-12-22    41.906776
2017-12-26    40.843605
2017-12-27    40.850784
2017-12-28    40.965721
2017-12-29    40.522728
Name: Close, dtype: float64
Date
2017-12-22     65397600
2017-12-26    132742000
2017-12-27     85992800
2017-12-28     65920800
2017-12-29    103999600
Name: Volume, dtype: int64


All the methods we applied to a Series index such as iloc[], loc[] and resampling methods, can also be applied to a DataFrame:

In [ ]:
aapl_2016 = df['2016']
aapl_month = aapl_2016.resample('M').agg(lambda x: x[-1])
print(aapl_month)

                 Open       High        Low  ...     Volume  Dividends  Stock Splits
Date                                         ...                                    
2016-01-31  21.852388  22.440250  21.750952  ...  257666000        0.0             0
2016-02-29  22.450753  22.768300  22.402079  ...  140865200        0.0             0
2016-03-31  25.431520  25.473242  25.236819  ...  103553600        0.0             0
2016-04-30  21.785531  21.954736  21.442489  ...  274126000        0.0             0
2016-05-31  23.226411  23.412969  23.044518  ...  169228800        0.0             0
2016-06-30  22.023115  22.333266  21.990468  ...  143345600        0.0             0
2016-07-31  24.296776  24.380727  24.177845  ...  110934800        0.0             0
2016-08-31  24.773059  24.986417  24.768369  ...  118649600        0.0             0
2016-09-30  26.367386  26.580745  26.212643  ...  145516400        0.0             0
2016-10-31  26.646394  26.782382  26.540886  ...  105677600      

We may select certain columns of a DataFrame using their names:

In [ ]:
aapl_bar = aapl_month[['Open', 'High', 'Low', 'Close']]
print(aapl_bar)

                 Open       High        Low      Close
Date                                                  
2016-01-31  21.852388  22.440250  21.750952  22.440250
2016-02-29  22.450753  22.768300  22.402079  22.411350
2016-03-31  25.431520  25.473242  25.236819  25.262316
2016-04-30  21.785531  21.954736  21.442489  21.727585
2016-05-31  23.226411  23.412969  23.044518  23.287043
2016-06-30  22.023115  22.333266  21.990468  22.293623
2016-07-31  24.296776  24.380727  24.177845  24.301439
2016-08-31  24.773059  24.986417  24.768369  24.876221
2016-09-30  26.367386  26.580745  26.212643  26.505718
2016-10-31  26.646394  26.782382  26.540886  26.620604
2016-11-30  26.300092  26.441490  25.986658  26.045574
2016-12-31  27.490191  27.619805  27.202681  27.294590


We can even specify both rows and columns using loc[]. The row indices and column names are separated by a comma:

In [ ]:
print(aapl_month.loc['2016-03':'2016-06',['Open', 'High', 'Low', 'Close']])

                 Open       High        Low      Close
Date                                                  
2016-03-31  25.431520  25.473242  25.236819  25.262316
2016-04-30  21.785531  21.954736  21.442489  21.727585
2016-05-31  23.226411  23.412969  23.044518  23.287043
2016-06-30  22.023115  22.333266  21.990468  22.293623


The subset methods in DataFrame is quite useful. By writing logical statements in square brackets, we can make customized subsets:

In [ ]:
import numpy as np

above = aapl_bar[aapl_bar.Close > np.mean(aapl_bar.Close)]
print(above)

                 Open       High        Low      Close
Date                                                  
2016-03-31  25.431520  25.473242  25.236819  25.262316
2016-08-31  24.773059  24.986417  24.768369  24.876221
2016-09-30  26.367386  26.580745  26.212643  26.505718
2016-10-31  26.646394  26.782382  26.540886  26.620604
2016-11-30  26.300092  26.441490  25.986658  26.045574
2016-12-31  27.490191  27.619805  27.202681  27.294590


## Data Validation
As mentioned, all methods that apply to a Series can also be applied to a DataFrame. Here we add a new column to an existing DataFrame:

In [ ]:
aapl_bar['rate_return'] = aapl_bar.Close.pct_change()
print(aapl_bar)

                 Open       High        Low      Close  rate_return
Date                                                               
2016-01-31  21.852388  22.440250  21.750952  22.440250          NaN
2016-02-29  22.450753  22.768300  22.402079  22.411350    -0.001288
2016-03-31  25.431520  25.473242  25.236819  25.262316     0.127211
2016-04-30  21.785531  21.954736  21.442489  21.727585    -0.139921
2016-05-31  23.226411  23.412969  23.044518  23.287043     0.071773
2016-06-30  22.023115  22.333266  21.990468  22.293623    -0.042660
2016-07-31  24.296776  24.380727  24.177845  24.301439     0.090062
2016-08-31  24.773059  24.986417  24.768369  24.876221     0.023652
2016-09-30  26.367386  26.580745  26.212643  26.505718     0.065504
2016-10-31  26.646394  26.782382  26.540886  26.620604     0.004334
2016-11-30  26.300092  26.441490  25.986658  26.045574    -0.021601
2016-12-31  27.490191  27.619805  27.202681  27.294590     0.047955


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Here the calculation introduced a NaN value. If the DataFrame is large, we would not be able to observe it. **isnull()** provides a convenient way to check abnormal values.

In [ ]:
missing = aapl_bar.isnull()
print(missing)
print('---------------------------------------------')
print(missing.describe())

             Open   High    Low  Close  rate_return
Date                                               
2016-01-31  False  False  False  False         True
2016-02-29  False  False  False  False        False
2016-03-31  False  False  False  False        False
2016-04-30  False  False  False  False        False
2016-05-31  False  False  False  False        False
2016-06-30  False  False  False  False        False
2016-07-31  False  False  False  False        False
2016-08-31  False  False  False  False        False
2016-09-30  False  False  False  False        False
2016-10-31  False  False  False  False        False
2016-11-30  False  False  False  False        False
2016-12-31  False  False  False  False        False
---------------------------------------------
         Open   High    Low  Close rate_return
count      12     12     12     12          12
unique      1      1      1      1           2
top     False  False  False  False       False
freq       12     12     12     12    

The row labelled "unique" indicates the number of unique values in each column. Since the "rate_return" column has 2 unique values, it has at least one missing value.

We can deduce the number of missing values by comparing "count" with "freq". There are 12 counts and 11 False values, so there is one True value which corresponds to the missing value.

We can also find the rows with missing values easily:

In [ ]:
print(missing[missing.rate_return == True])

             Open   High    Low  Close  rate_return
Date                                               
2016-01-31  False  False  False  False         True


Usually when dealing with missing data, we either delete the whole row or fill it with some value. As we introduced in the Series chapter, the same method **dropna()** and **fillna()** can be applied to a DataFrame.

In [ ]:
drop = aapl_bar.dropna()
print(drop)
print('\n--------------------------------------------------\n')
fill = aapl_bar.fillna(0)
print(fill)

                 Open       High        Low      Close  rate_return
Date                                                               
2016-02-29  22.450753  22.768300  22.402079  22.411350    -0.001288
2016-03-31  25.431520  25.473242  25.236819  25.262316     0.127211
2016-04-30  21.785531  21.954736  21.442489  21.727585    -0.139921
2016-05-31  23.226411  23.412969  23.044518  23.287043     0.071773
2016-06-30  22.023115  22.333266  21.990468  22.293623    -0.042660
2016-07-31  24.296776  24.380727  24.177845  24.301439     0.090062
2016-08-31  24.773059  24.986417  24.768369  24.876221     0.023652
2016-09-30  26.367386  26.580745  26.212643  26.505718     0.065504
2016-10-31  26.646394  26.782382  26.540886  26.620604     0.004334
2016-11-30  26.300092  26.441490  25.986658  26.045574    -0.021601
2016-12-31  27.490191  27.619805  27.202681  27.294590     0.047955

--------------------------------------------------

                 Open       High        Low      Close  rate_re

## DataFrame Concat
We have seen how to extract a Series from a dataFrame. Now we need to consider how to merge a Series or a DataFrame into another one.

In Pandas, the function **concat()** allows us to merge multiple Series into a DataFrame:

In [ ]:
s1 = pd.Series([143.5, 144.09, 142.73, 144.18, 143.77], name = 'AAPL')
s2 = pd.Series([898.7, 911.71, 906.69, 918.59, 926.99], name = 'GOOG')
data_frame = pd.concat([s1,s2], axis = 1)
print(data_frame)

     AAPL    GOOG
0  143.50  898.70
1  144.09  911.71
2  142.73  906.69
3  144.18  918.59
4  143.77  926.99


The "axis = 1" parameter will join two DataFrames by columns:

In [ ]:
log_price = np.log(aapl_bar.Close)
log_price.name = 'log_price'
print(log_price)
print('\n---------------------- separate line--------------------\n')
concat = pd.concat([aapl_bar, log_price], axis = 1)
print(concat)

Date
2016-01-31    3.110856
2016-02-29    3.109568
2016-03-31    3.229314
2016-04-30    3.078583
2016-05-31    3.147897
2016-06-30    3.104301
2016-07-31    3.190536
2016-08-31    3.213912
2016-09-30    3.277360
2016-10-31    3.281685
2016-11-30    3.259848
2016-12-31    3.306689
Freq: M, Name: log_price, dtype: float64

---------------------- separate line--------------------

                 Open       High        Low      Close  rate_return  log_price
Date                                                                          
2016-01-31  21.852388  22.440250  21.750952  22.440250          NaN   3.110856
2016-02-29  22.450753  22.768300  22.402079  22.411350    -0.001288   3.109568
2016-03-31  25.431520  25.473242  25.236819  25.262316     0.127211   3.229314
2016-04-30  21.785531  21.954736  21.442489  21.727585    -0.139921   3.078583
2016-05-31  23.226411  23.412969  23.044518  23.287043     0.071773   3.147897
2016-06-30  22.023115  22.333266  21.990468  22.293623    -0.04266

We can also join two DataFrames by rows. Consider these two DataFrames:

In [ ]:
df_volume = aapl_table.loc['2016-10':'2017-04',['Volume', 'Stock Splits']].resample('M').agg(lambda x: x[-1])
print(df_volume)
print('\n---------------------- separate line--------------------\n')
df_2017 = aapl_table.loc['2016-10':'2017-04',['Open', 'High', 'Low', 'Close']].resample('M').agg(lambda x: x[-1])
print(df_2017)

               Volume  Stock Splits
Date                               
2016-10-31  105677600             0
2016-11-30  144649200             0
2016-12-31  122345200             0
2017-01-31  196804000             0
2017-02-28   93931600             0
2017-03-31   78646800             0
2017-04-30   83441600             0

---------------------- separate line--------------------

                 Open       High        Low      Close
Date                                                  
2016-10-31  26.646394  26.782382  26.540886  26.620604
2016-11-30  26.300092  26.441490  25.986658  26.045574
2016-12-31  27.490191  27.619805  27.202681  27.294590
2017-01-31  28.550684  28.607243  28.425783  28.597816
2017-02-28  32.444877  32.530084  32.354935  32.423576
2017-03-31  34.016455  34.146633  33.848407  34.002254
2017-04-30  34.104032  34.153737  33.909951  33.999889


Now we merge the DataFrames with our DataFrame 'aapl_bar'

In [ ]:
concat = pd.concat([aapl_bar, df_volume], axis = 1)
print(concat)

                 Open       High  ...       Volume  Stock Splits
Date                              ...                           
2016-01-31  21.852388  22.440250  ...          NaN           NaN
2016-02-29  22.450753  22.768300  ...          NaN           NaN
2016-03-31  25.431520  25.473242  ...          NaN           NaN
2016-04-30  21.785531  21.954736  ...          NaN           NaN
2016-05-31  23.226411  23.412969  ...          NaN           NaN
2016-06-30  22.023115  22.333266  ...          NaN           NaN
2016-07-31  24.296776  24.380727  ...          NaN           NaN
2016-08-31  24.773059  24.986417  ...          NaN           NaN
2016-09-30  26.367386  26.580745  ...          NaN           NaN
2016-10-31  26.646394  26.782382  ...  105677600.0           0.0
2016-11-30  26.300092  26.441490  ...  144649200.0           0.0
2016-12-31  27.490191  27.619805  ...  122345200.0           0.0
2017-01-31        NaN        NaN  ...  196804000.0           0.0
2017-02-28        NaN    

By default the DataFrame are joined with all of the data. This default options results in zero information loss. We can also merge them by intersection, this is called 'inner join

In [ ]:
concat = pd.concat([aapl_bar,df_volume],axis = 1, join = 'inner')
print(concat)

                 Open       High  ...     Volume  Stock Splits
Date                              ...                         
2016-10-31  26.646394  26.782382  ...  105677600             0
2016-11-30  26.300092  26.441490  ...  144649200             0
2016-12-31  27.490191  27.619805  ...  122345200             0

[3 rows x 7 columns]


Only the intersection part was left if use 'inner join' method. Now let's try to append a DataFrame to another one:

In [ ]:
append = aapl_bar.append(df_2017)
print(append)

                 Open       High        Low      Close  rate_return
Date                                                               
2016-01-31  21.852388  22.440250  21.750952  22.440250          NaN
2016-02-29  22.450753  22.768300  22.402079  22.411350    -0.001288
2016-03-31  25.431520  25.473242  25.236819  25.262316     0.127211
2016-04-30  21.785531  21.954736  21.442489  21.727585    -0.139921
2016-05-31  23.226411  23.412969  23.044518  23.287043     0.071773
2016-06-30  22.023115  22.333266  21.990468  22.293623    -0.042660
2016-07-31  24.296776  24.380727  24.177845  24.301439     0.090062
2016-08-31  24.773059  24.986417  24.768369  24.876221     0.023652
2016-09-30  26.367386  26.580745  26.212643  26.505718     0.065504
2016-10-31  26.646394  26.782382  26.540886  26.620604     0.004334
2016-11-30  26.300092  26.441490  25.986658  26.045574    -0.021601
2016-12-31  27.490191  27.619805  27.202681  27.294590     0.047955
2016-10-31  26.646394  26.782382  26.540886  26.

'Append' is essentially to concat two DataFrames by axis = 0, thus here is an alternative way to append:

In [ ]:
concat = pd.concat([aapl_bar, df_2017], axis = 0)
print(concat)

                 Open       High        Low      Close  rate_return
Date                                                               
2016-01-31  21.852388  22.440250  21.750952  22.440250          NaN
2016-02-29  22.450753  22.768300  22.402079  22.411350    -0.001288
2016-03-31  25.431520  25.473242  25.236819  25.262316     0.127211
2016-04-30  21.785531  21.954736  21.442489  21.727585    -0.139921
2016-05-31  23.226411  23.412969  23.044518  23.287043     0.071773
2016-06-30  22.023115  22.333266  21.990468  22.293623    -0.042660
2016-07-31  24.296776  24.380727  24.177845  24.301439     0.090062
2016-08-31  24.773059  24.986417  24.768369  24.876221     0.023652
2016-09-30  26.367386  26.580745  26.212643  26.505718     0.065504
2016-10-31  26.646394  26.782382  26.540886  26.620604     0.004334
2016-11-30  26.300092  26.441490  25.986658  26.045574    -0.021601
2016-12-31  27.490191  27.619805  27.202681  27.294590     0.047955
2016-10-31  26.646394  26.782382  26.540886  26.

Please note that if the two DataFrame have some columns with the same column names, these columns are considered to be the same and will be merged. It's very important to have the right column names. If we change a column names here:

In [ ]:
df_2017.columns = ['Change', 'High','Low','Close']
concat = pd.concat([aapl_bar, df_2017], axis = 0)
print(concat)

                 Open       High        Low      Close  rate_return     Change
Date                                                                          
2016-01-31  21.852388  22.440250  21.750952  22.440250          NaN        NaN
2016-02-29  22.450753  22.768300  22.402079  22.411350    -0.001288        NaN
2016-03-31  25.431520  25.473242  25.236819  25.262316     0.127211        NaN
2016-04-30  21.785531  21.954736  21.442489  21.727585    -0.139921        NaN
2016-05-31  23.226411  23.412969  23.044518  23.287043     0.071773        NaN
2016-06-30  22.023115  22.333266  21.990468  22.293623    -0.042660        NaN
2016-07-31  24.296776  24.380727  24.177845  24.301439     0.090062        NaN
2016-08-31  24.773059  24.986417  24.768369  24.876221     0.023652        NaN
2016-09-30  26.367386  26.580745  26.212643  26.505718     0.065504        NaN
2016-10-31  26.646394  26.782382  26.540886  26.620604     0.004334        NaN
2016-11-30  26.300092  26.441490  25.986658  26.0455

Since the column name of 'Open' has been changed, the new DataFrame has an new column named 'Change'.

# Summary

Hereby we introduced the most import part of python: resampling and DataFrame manipulation. We only introduced the most commonly used method in Financial data analysis. There are also many methods used in data mining, which are also beneficial. You can always check the [Pandas](https://pandas.pydata.org/pandas-docs/stable/index.html) official documentations for help.